In [25]:
# AutoEncoders

# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

# Importing the dataset
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

# Preparing the training set and the test set
training_set = np.array(pd.read_csv('ml-100k/u1.base', delimiter='\t'), dtype=int)
test_set = np.array(pd.read_csv('ml-100k/u1.test', delimiter='\t'), dtype=int)

# Getting the number of users and movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0]))) # must check both traing and test set for max number
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))# must check both traing and test set for max number


# create the matrix of users = lines/rows and movies = col/features
def convert(data):
    new_data = []
    for id_users in range(1,nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users ] # get all the movies by the user_id
        id_rating = data[:,2][data[:,0] == id_users ] # get all the rating by the user_id
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_rating
        new_data.append(list(ratings))
    return new_data

# convert and change to Torch Tensors
training_set = torch.FloatTensor(convert(training_set))
test_set = torch.FloatTensor(convert(test_set))